In [37]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms.v2 as transforms
from torchvision.models.video import r2plus1d_18, R2Plus1D_18_Weights
from os import path

from video_tensor_dataset import VideoTensorDataset
from train_model import TrainModel

BATCH_SIZE = 4
NUM_EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device {DEVICE}")

ImportError: cannot import name 'ConvertBCHWToCBHW' from 'torchvision.transforms' (c:\Users\Nitzan\miniconda3\envs\DL_env\Lib\site-packages\torchvision\transforms\__init__.py)

In [ ]:
MY_NAME_IS = 'Nitzan'

DATA_FOLDER_DICT = {
    'Victor': ['E:\DeepFakeDetection\dfdc_train_all','E:\DeepFakeDetection\smalldata'],
    'Nitzan': ['D:\dfdc','D:\dfdc_small2'],
    'Netanel':['F:\input','F:\input']
  }

BIG_DATA_FOLDER, SMALL_DATA_FOLDER = DATA_FOLDER_DICT[MY_NAME_IS]

TRAIN_PARTS = [0, 1, 2, 3]
VALIDATION_PARTS = [4]

In [ ]:
model = r2plus1d_18(weights=R2Plus1D_18_Weights.DEFAULT)

In [ ]:
# Replace head

model.fc = nn.Linear(model.fc.in_features, 2)

# Freeze all layers except the new head
for key, params in model.named_parameters():
    if 'fc' not in key:
        params.requires_grad = False


print(model) # Perhaps we should also print torchinfo.summary here

VideoResNet(
  (stem): R2Plus1dStem(
    (0): Conv3d(3, 45, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
    (1): BatchNorm3d(45, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv3d(45, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
    (4): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv2Plus1D(
          (0): Conv3d(64, 144, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
          (1): BatchNorm3d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv3d(144, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
        )
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [42]:
# Taken from https://github.com/pytorch/vision/tree/main/references/video_classification
mean = [0.43216, 0.394666, 0.37645]
std = [0.22803, 0.22145, 0.216989]

# Initialize train and validation datasets
# TODO videodataset root paths and transforms - this depends on Victor's offline preprocessing 
train_roots = [path.join(SMALL_DATA_FOLDER, str(i)) for i in TRAIN_PARTS]
train_transform = transforms.Compose([
    transforms.ConvertImageDtype(torch.float32),
    # transforms.Lambda(lambda video: torch.narrow(video, 0, 0, 90)),
    transforms.Normalize(mean, std),
    # transforms.Lambda(lambda video: video.permute(3, 0, 1, 2))
])

train_ds = VideoTensorDataset(
  original_data_path=BIG_DATA_FOLDER, 
  tensor_data_paths=train_roots, 
  transform=train_transform
)

validation_roots = [path.join(SMALL_DATA_FOLDER, str(i)) for i in VALIDATION_PARTS]
# validation_transform = Compose([])

validation_ds = VideoTensorDataset(
  original_data_path=BIG_DATA_FOLDER, 
  tensor_data_paths=validation_roots, 
  # transform=validation_transform
)

# Initialize dataloaders

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
validation_dl = DataLoader(validation_ds, batch_size=BATCH_SIZE, shuffle=False)

In [43]:
batch, labels = next(iter(train_dl))
print(batch.shape, labels.shape)

RuntimeError: The size of tensor a (54) must match the size of tensor b (3) at non-singleton dimension 1

In [23]:
# implementation #1
from sklearn.metrics import matthews_corrcoef
def mcc(y_true, y_pred):
    return matthews_corrcoef(y_true, y_pred)

# implementation #2
import torch
def matthews_corrcoef(y_true, y_pred):
    y_pred_pos = torch.round(torch.sigmoid(y_pred))
    y_pred_neg = 1 - y_pred_pos

    y_true_pos = y_true
    y_true_neg = 1 - y_true

    tp = (y_true_pos * y_pred_pos).sum().to(torch.float32)
    tn = (y_true_neg * y_pred_neg).sum().to(torch.float32)
    fp = (y_true_neg * y_pred_pos).sum().to(torch.float32)
    fn = (y_true_pos * y_pred_neg).sum().to(torch.float32)

    numerator = (tp * tn) - (fp * fn)
    denominator = torch.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + 1e-8)

In [24]:
from torchvision.ops import sigmoid_focal_loss
from sklearn.metrics import matthews_corrcoef

optimizer = torch.optim.Adam(model.parameters(), lr=3e-4) # TODO arguments for the optimizer
def loss_fn(y_pred, y_true):
    return sigmoid_focal_loss(y_pred, y_true, gamma=2.0, weights=[5/6, 1/6], reduction='mean')
score_fn = matthews_corrcoef

In [25]:
# Train the model!

model, train_loss, train_score, validation_loss, validation_score, _ = TrainModel(
  model, 
  train_dl, 
  validation_dl, 
  optimizer, 
  NUM_EPOCHS, 
  loss_fn, 
  score_fn
)

RuntimeError: expected scalar type Byte but found Float

In [ ]:
# TODO plot the results?